# BLP extension with product embeddings

code adapted from [wangzhegeek](https://github.com/wangzhegeek/EGES/tree/master), using Nevo (2000) fake cereal data.
> Nevo, A. (2000), A Practitioner's Guide to Estimation of Random-Coefficients Logit Models of Demand. Journal of Economics & Management Strategy, 9: 513-548.

## Get embeddings
### Import libraries

In [2]:
# environment: python 3.6.x, tensorflow==1.14.0

import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import LabelEncoder
import networkx as nx
from joblib import Parallel, delayed
import itertools
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

### Create some tools first

In [3]:
# alias sampling
def create_alias_table(area_ratio):
    '''
    area_ratio: list, elements add up to 1, the probability distribution to sample from
    '''
    l = len(area_ratio)
    area_ratio = [prop*l for prop in area_ratio]
    accept, alias = [0]*l, [0]*l
    small, large = [], []
    for i, prob in enumerate(area_ratio):
        if prob < 1.0:
            small.append(i)
        else:
            large.append(i)
            
    while small and large:
        small_index, large_index = small.pop(), large.pop()
        accept[small_index] = area_ratio[small_index]
        alias[small_index] = large_index
        area_ratio[large_index] = area_ratio[large_index] - (1-area_ratio[small_index])
        if area_ratio[large_index] < 1.0:
            small.append(large_index)
        else:
            large.append(large_index)
    while large:
        large_index = large.pop()
        accept[large_index] = 1
    while small:
        small_index = small.pop()
        accept[small_index] = 1
    return accept, alias

def alias_sample(accept, alias):
    N = len(accept)
    i = int(np.random.random()*N)
    r = np.random.random()
    if r < accept[i]:
        return i
    else:
        return alias[i]

In [4]:
def partition_num(num, workers):
    if num % workers == 0:
        return [num//workers]*workers
    else:
        return [num//workers]*workers + [num%workers]
    
def graph_context_batch_iter(all_pairs, batch_size, side_info, num_attr):
    while True:
        start_idx = np.random.randint(0, len(all_pairs)-batch_size)
        batch_idx = np.array(range(start_idx, start_idx+batch_size))
        batch_idx = np.random.permutation(batch_idx)
        batch = np.zeros((batch_size, num_attr),dtype=np.int32)
        labels = np.zeros((batch_size,1),dtype=np.int32)
        batch[:] = side_info[all_pairs[batch_idx,0]]
        labels[:,0] = all_pairs[batch_idx,1]
        yield batch, labels
        
def write_embedding(embedding_result, output_filename):
    f = open(output_filename,'w')
    for i in range(len(embedding_result)):
        s = ",".join(str(f) for f in embedding_result[i].tolist())
        f.write(s + '\n')
    f.close()
    return

####### TODO: Adapt this #######
def plot_embedding(embed_mat, side_info_mat):
    model = TSNE(n_components=2)
    node_pos = model.fit_transform(embed_mat)
    # suppose there are 3 product attributes
    attr1_idx, attr2_idx, attr3_idx = {}, {}, {}
    for i in range(len(node_pos)):
        attr1_idx.setdefault(side_info_mat[i,1],[])
        attr1_idx[side_info_mat[i,1]].append(i)
        attr2_idx.setdefault(side_info_mat[i,2],[])
        attr2_idx[side_info_mat[i,2]].append(i)
        attr3_idx.setdefault(side_info_mat[i,3],[])
        attr3_idx[side_info_mat[i,3]].append(i)
    
    plt.figure()
    for c, idx in attr1_idx.items():
        plt.scatter(node_pos[idx,0], node_pos[idx,1], label=c)
    plt.title('Attribute 1 distribution')
    plt.savefig(r"E:\学习\UBC\ECON622\final project\jdata_cache\attr1.png")
    
    plt.figure()
    for c, idx in attr2_idx.items():
        plt.scatter(node_pos[idx,0], node_pos[idx,1], label=c)
    plt.title('Attribute 2 distribution')
    plt.savefig(r"E:\学习\UBC\ECON622\final project\jdata_cache\attr2.png")

    plt.figure()
    for c, idx in attr3_idx.items():
        plt.scatter(node_pos[idx,0], node_pos[idx,1], label=c)
    plt.title('Attribute 3 distribution')
    plt.savefig(r"E:\学习\UBC\ECON622\final project\jdata_cache\attr3.png")
    return
#################################

### Random Walk

In [5]:
class RandomWalker:
    
    # deepwalk performs first order random walk
    # node2vec performs second order random walk, which incorporates information from the previous node.
    
    def __init__(self,G,p=1,q=1):
        """
        G: the graph
        p: return parameter. p controls the likelihood of immediately returning to a node we just visited.
        q: in-out parameter. q controls how likely we are to stay in the neighborhood of a certain node, 
                                or are we more likely to visit nodes further away from this node.
        (when p=q=1, it becomes first order random walk/1-hop transition)
        G.neighbors(k): Returns an iterator over all neighbors of node k
        """
        self.G = G
        self.p = p
        self.q = q
        
    def deepwalk_walk(self,walk_length,start_node):
        
        walk = [start_node]
        
        while len(walk) < walk_length:
            cur = walk[-1] # current node
            cur_nbrs = list(self.G.neighbors(cur)) # current node neighbors
            if len(cur_nbrs) > 0:
                walk.append(random.choice(cur_nbrs))
            else:
                break
        return walk
    
    def node2vec_walk(self, walk_length, start_node):
        G = self.G
        alias_nodes = self.alias_nodes
        alias_edges = self.alias_edges
        
        walk = [start_node]
        
        while len(walk) < walk_length:
            cur = walk[-1]
            cur_nbrs = list(self.G.neighbors(cur))
            if len(cur_nbrs) > 0:
                if len(walk) == 1:
                    walk.append(cur_nbrs[alias_sample(alias_nodes[cur][0],alias_nodes[cur][1])])
                else:
                    prev = walk[-2]
                    edge = (prev, cur)
                    next_node = cur_nbrs[alias_sample(alias_edges[edge][0],alias_edges[edge][1])]
                    walk.append(next_node)
            else:
                break
        return walk
    
    def simulate_walks(self, num_walks, walk_length, workers=1, verbose=0):
        G = self.G
        nodes = list(G.nodes())
        results = Parallel(n_jobs=workers,verbose=verbose,)(delayed(self._simulate_walks)(nodes, num, walk_length) for num in partition_num(num_walks,workers))
        walks = list(itertools.chain(*results))
        return walks
    
    def _simulate_walks(self, nodes, num_walks, walk_length,):
        walks = []
        for _ in range(num_walks):
            random.shuffle(nodes)
            for v in nodes:
                if self.p == 1 and self.q == 1:
                    walks.append(self.deepwalk_walk(walk_length=walk_length, start_node=v))
                else:
                    walks.append(self.node2vec_walk(walk_length=walk_length, start_node=v))
        return walks
    
    def get_alias_edge(self, t, v):
        '''
        get the alias table for node2vec_walk
        '''
        G = self.G
        p = self.p
        q = self.q
        
        unnormalized_probs = []
        for x in G.neighbors(v):
            weight = G[v][x].get('weight',1.0)
            if x == t:
                unnormalized_probs.append(weight/p)
            elif G.has_edge(x,t):
                unnormalized_probs.append(weight)
            else:
                unnormalized_probs.append(weight/q)
        norm_const = sum(unnormalized_probs)
        normalized_probs = [float(u_prob)/norm_const for u_prob in unnormalized_probs]
        
        alias_edge = create_alias_table(normalized_probs)
        return alias_edge
    
    def preprocess_transition_probs(self):
        
        G =  self.G
        alias_nodes = {}
        for node in G.nodes():
            unnormalized_probs = [G[node][nbr].get('weight',1.0) for nbr in G.neighbors(node)]
            norm_const = sum(unnormalized_probs)
            normalized_probs = [float(u_prob)/norm_const for u_prob in unnormalized_probs]
            alias_nodes[node] = create_alias_table(normalized_probs)
        
        alias_edges = {}
        for edge in G.edges():
            alias_edges[edge] = self.get_alias_edge(edge[0],edge[1])
        
        self.alias_nodes = alias_nodes
        self.alias_edges = alias_edges
        
        return

### Process the raw data

In [11]:
def process_nevo_data():
    '''
    recover user history from agent_data:
        (1) recover each consumer's choice in each market
        (2) assume consumers with the same {income, age, child} are the same people, give them a unique user_id
    '''
    
    # recover choices
    product_data = pd.read_csv(r"E:\学习\UBC\ECON622\final project\nevodata\product_data.csv")
    agent_data = pd.read_csv(r"E:\学习\UBC\ECON622\final project\nevodata\agent_data.csv")

    product = product_data[['market_ids', 'product_ids', 'prices', 'sugar', 'mushy']].copy()
    product.insert(2, 'constant', np.ones(product.shape[0]))
    agent = agent_data[['market_ids','nodes0','nodes1','nodes2','nodes3','income','income_squared','age','child']].copy()

    # variable order: constant, price, sugar, mushy; income, income_squared, age, child 
    beta = np.array([-1.841, -32.433, 0.148, 0.788])
    sigma = np.diag([0.377, 1.848, 0.004, 0.081])
    pi = np.array([
      [ 3.089,  0,      1.186,  0     ],
      [16.598, -0.659,  0,      11.625],
      [-0.193,  0,      0.029,  0     ],
      [ 1.468,  0,     -1.514,  0     ]
    ])
    
    market_list = product['market_ids'].unique()
    for market in market_list:
        sub_agent = agent[agent['market_ids']==market]
        sub_product = product[product['market_ids']==market]
        choice = []
        for i in range(sub_agent.shape[0]):
            v_list = []
            for j in range(sub_product.shape[0]):
                mu = np.dot((np.dot(sigma,sub_agent.iloc[i,1:5]) + np.dot(pi,sub_agent.iloc[i,5:9])),sub_product.iloc[j,2:6])
                delta = np.dot(beta,sub_product.iloc[j,2:6])
                v_list.append(mu+delta)
            choice_idx = (np.exp(v_list)/np.sum(np.exp(v_list))).argmax()
            choice_product = sub_product.iloc[choice_idx,1]
            choice.append(choice_product)
        agent.loc[agent['market_ids']==market,'product_ids'] = choice
        
    
    # assign user_id
    agent['user_id'] = agent.apply(lambda row: hash((row['age'], row['child'], row['income'])), axis=1)
    agent['user_id'] = agent['user_id'].abs()
    action_data = agent[['user_id','product_ids']].copy()
    
    return action_data

In [17]:
def get_session(action_data, use_type=None):
    '''
    action_data: agent history recordings
    returns a list of sessions, each session contains a series of product_ids
    '''
    action_data = action_data.sort_values(by=['user_id'],ascending=True)
    group_action_data = action_data.groupby('user_id').agg(list)
    session_list = group_action_data['product_ids'].to_list()
    return session_list

def get_graph_context_all_pairs(walks,window_size):
    all_pairs = []
    for k in range(len(walks)):
        for i in range(len(walks[k])):
            for j in range(i-window_size,i+window_size+1):
                if i==j or j<0 or j>=len(walks[k]):
                    continue
                else:
                    all_pairs.append([walks[k][i],walks[k][j]])
    return np.array(all_pairs,dtype=np.int32)
   

In [39]:
def data_process(p=0.25,q=2,num_walks=10,walk_length=10,window_size=5):
    
    # STEP 1: get user sessions from user history file

    action_data = process_nevo_data()
    all_skus = action_data['product_ids'].unique()
    all_skus = pd.DataFrame({'product_ids':list(all_skus)})
    sku_lbe = LabelEncoder()
    all_skus['product_ids'] = sku_lbe.fit_transform(all_skus['product_ids'])
    action_data['product_ids'] = sku_lbe.transform(action_data['product_ids'])
    
    # collect sessions from all users
    session_list = get_session(action_data)
    session_list_all = []
    for item_list in session_list:
        if len(item_list) > 1: # single node is non-informative
            session_list_all.append(item_list)
    
    # session to (directed and weighted) graph, save temp results
    node_pair = dict()
    for session in session_list_all:
        for i in range(1, len(session)):
            if (session[i-1],session[i]) not in node_pair.keys():
                node_pair[(session[i-1], session[i])] = 1
            else:
                node_pair[(session[i-1], session[i])] += 1
                
    in_node_list = list(map(lambda x:x[0], list(node_pair.keys()))) # in node -> out node
    out_node_list = list(map(lambda x:x[1], list(node_pair.keys())))
    weight_list = list(node_pair.values())
    graph_df = pd.DataFrame({'in_node': in_node_list,'out_node':out_node_list,'weight':weight_list})
    graph_df.to_csv(r"E:\学习\UBC\ECON622\final project\nevodata_cache\graph.csv",sep=" ",index=False,header=False)
    
    G = nx.read_edgelist(r"E:\学习\UBC\ECON622\final project\nevodata_cache\graph.csv", 
                         create_using=nx.DiGraph(), 
                         nodetype=None, 
                         data=[('weight',int)])
    walker = RandomWalker(G, p=p, q=q)
    walker.preprocess_transition_probs()
    
    session_reproduce = walker.simulate_walks(num_walks=num_walks,walk_length=walk_length,workers=4,verbose=1)
    session_reproduce = list(filter(lambda x: len(x)>2, session_reproduce))

    # STEP 2: get side information from product attribute file
    product_data = pd.read_csv(r"E:\学习\UBC\ECON622\final project\nevodata\product_data.csv")
    product = product_data[['product_ids', 'sugar', 'mushy']].copy() # ignore price for now
    product.drop_duplicates(inplace=True)
    all_skus['product_ids'] = sku_lbe.inverse_transform(all_skus['product_ids'])
    sku_side_info = pd.merge(all_skus, product, on='product_ids',how='left').fillna(0)
    
    for product_attr in sku_side_info.columns:
        if product_attr != 'product_ids':
            lbe = LabelEncoder()
            sku_side_info[product_attr] = lbe.fit_transform(sku_side_info[product_attr])
        else:
            sku_side_info[product_attr] = sku_lbe.transform(sku_side_info[product_attr])
            
    sku_side_info = sku_side_info.sort_values(by=['product_ids'],ascending=True)
    sku_side_info.to_csv(r"E:\学习\UBC\ECON622\final project\nevodata_cache\sku_side_info.csv",index=False,header=False,sep=" ")
    
    # get pair
    all_pairs = get_graph_context_all_pairs(session_reproduce,window_size)
    np.savetxt(r"E:\学习\UBC\ECON622\final project\nevodata_cache\all_pairs.txt",X=all_pairs,fmt='%d',delimiter=" ")
 
    return

In [40]:
data_process()

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.3s finished


### Enhanced Graph Embedding with Side information (EGES) model

In [19]:
class EGES_Model:
    def __init__(self, num_nodes, num_attr, attr_lens, n_sampled=100, embedding_dim=128, lr=0.001):
        '''
        num_attr: number of product attributes + 1, which is the product itself (as one embedding)
        attr_lens: a list of attribute lengths. Each attribute is represented by a one-hot vector, 
            the dimension/length depends on how many unique values are there for this attribute.
        '''
        self.n_sampled = n_sampled
        self.num_attr = num_attr
        self.attr_lens = attr_lens
        self.embedding_dim = embedding_dim
        self.num_nodes = num_nodes
        self.lr = lr
        self.softmax_w = tf.Variable(tf.truncated_normal((num_nodes,embedding_dim),stddev=0.1),name='softmax_w')
        self.softmax_b = tf.Variable(tf.zeros(num_nodes),name='softmax_b')
        self.inputs = self.input_init()
        self.embedding = self.embedding_init()
        self.alpha_embedding = tf.Variable(tf.random_uniform((num_nodes,num_attr),-1,1))
        self.merge_emb = self.attention_merge()
        self.cost = self.make_skipgram_loss()
        self.train_op = tf.train.AdamOptimizer(lr).minimize(self.cost)
        
    def embedding_init(self):
        '''
        each product attribute has its own embedding
        '''
        embedding_collection = []
        for i in range(self.num_attr):
            embedding_var = tf.Variable(tf.random_uniform((self.attr_lens[i],self.embedding_dim),-1,1), name='embedding'+str(i),trainable=True)
            embedding_collection.append(embedding_var)
        return embedding_collection
    
    def attention_merge(self):
        embed_list = []
        num_embed_list = []
        for i in range(self.num_attr):
            cat_embed = tf.nn.embedding_lookup(self.embedding[i],self.inputs[i])
            embed_list.append(cat_embed)
        stack_embed = tf.stack(embed_list, axis=-1)
        # attention merge
        alpha_embed = tf.nn.embedding_lookup(self.alpha_embedding, self.inputs[0])
        alpha_embed_expand = tf.expand_dims(alpha_embed, 1)
        merge_emb = tf.reduce_sum(stack_embed * tf.exp(alpha_embed_expand),axis=-1) / tf.reduce_sum(tf.exp(alpha_embed_expand),axis=-1)
        return merge_emb
    
    def input_init(self):
        '''
        initialize attribute inputs, label stored in the last element
        '''
        input_list = []
        for i in range(self.num_attr):
            input_col = tf.placeholder(tf.int32,[None],name='inputs_'+str(i))
            input_list.append(input_col)
        input_list.append(tf.placeholder(tf.int32,shape=[None,1],name='label'))
        return input_list
    
    def make_skipgram_loss(self):
        '''
        sampled softmax loss: faster when there are huge number of classes, for training purpose only
        '''
        loss = tf.reduce_mean(tf.nn.sampled_softmax_loss(
            weights=self.softmax_w,
            biases=self.softmax_b,
            labels=self.inputs[-1],
            inputs=self.merge_emb,
            num_sampled=self.n_sampled,
            num_classes=self.num_nodes,
            num_true=1,
            sampled_values=tf.random.uniform_candidate_sampler(
                true_classes=tf.cast(self.inputs[-1],tf.int64),
                num_true=1,
                num_sampled=self.n_sampled,
                unique=True,
                range_max=self.num_nodes
            )
        ))
        
        return loss

### Train the model and get the embeddings

In [68]:
def run_EGES(output_path,batch_size=2048,n_sampled=10,epochs=2,lr=0.001,num_attr=3,embedding_dim=128,if_plot=False):
    
    side_info = np.loadtxt(r"E:\学习\UBC\ECON622\final project\nevodata_cache\sku_side_info.csv",dtype=np.int32,delimiter=" ")
    all_pairs = np.loadtxt(r"E:\学习\UBC\ECON622\final project\nevodata_cache\all_pairs.txt",dtype=np.int32,delimiter=" ")
    attr_lens = []
    for i in range(side_info.shape[1]):
        tmp_len = len(set(side_info[:,i]))
        attr_lens.append(tmp_len)
    
    num_nodes = len(side_info)
    EGES = EGES_Model(num_nodes,num_attr,attr_lens,n_sampled,embedding_dim,lr)
    
    # initialize model
    init = tf.global_variables_initializer()
    config_tf = tf.ConfigProto()
    config_tf.gpu_options.allow_growth = True
    sess = tf.Session(config=config_tf)
    sess.run(init)
    
    print_every_k_iterations = 100
    loss = 0
    iteration = 0
    
    max_iter = len(all_pairs) // batch_size * epochs
    for each_iter in range(max_iter):
        iteration += 1
        batch_features, batch_labels = next(graph_context_batch_iter(all_pairs,batch_size,side_info,num_attr))
        feed_dict = {input_col: batch_features[:,i] for i,input_col in enumerate(EGES.inputs[:-1])}
        feed_dict[EGES.inputs[-1]] = batch_labels
        _, train_loss = sess.run([EGES.train_op,EGES.cost],feed_dict=feed_dict)
        loss += train_loss
        
        if iteration % print_every_k_iterations == 0:
            e = iteration * batch_size//len(all_pairs)
            print('Epoch {}/{}'.format(e,epochs),
                  'Iteration: {}'.format(iteration), 
                  'Average training loss: {:.4f}'.format(loss/print_every_k_iterations)
                  )
            loss = 0
    print('optimization finished! saving results...')
    saver = tf.train.Saver()
    saver.save(sess,"E:\\学习\\UBC\\ECON622\\final project\\nevodata_cache\\checkpoints\\EGES")
    

    feed_dict_test = {input_col: list(side_info[:,i]) for i,input_col in enumerate(EGES.inputs[:-1])}
    feed_dict_test[EGES.inputs[-1]] = np.zeros((len(side_info),1),dtype=np.int32)
    embedding_result = sess.run(EGES.merge_emb,feed_dict=feed_dict_test)
    write_embedding(embedding_result, output_path)
    
    # plot the result
    if if_plot == True:
        print('visualization...')
        plot_embeddings(embedding_result[:5000,:],side_info[:5000,:])
    
    return

In [69]:
out = r"E:\学习\UBC\ECON622\final project\nevodata_cache\eges.embed"
run_EGES(out,batch_size=16,n_sampled=1,epochs=4,embedding_dim=16)

Epoch 0/4 Iteration: 100 Average training loss: 0.6336
Epoch 0/4 Iteration: 200 Average training loss: 0.5623
Epoch 0/4 Iteration: 300 Average training loss: 0.5242
Epoch 0/4 Iteration: 400 Average training loss: 0.5078
Epoch 0/4 Iteration: 500 Average training loss: 0.4678
Epoch 0/4 Iteration: 600 Average training loss: 0.4512
Epoch 1/4 Iteration: 700 Average training loss: 0.4013
Epoch 1/4 Iteration: 800 Average training loss: 0.3782
Epoch 1/4 Iteration: 900 Average training loss: 0.3874
Epoch 1/4 Iteration: 1000 Average training loss: 0.3527
Epoch 1/4 Iteration: 1100 Average training loss: 0.3547
Epoch 1/4 Iteration: 1200 Average training loss: 0.3125
Epoch 1/4 Iteration: 1300 Average training loss: 0.3088
Epoch 2/4 Iteration: 1400 Average training loss: 0.3030
Epoch 2/4 Iteration: 1500 Average training loss: 0.2768
Epoch 2/4 Iteration: 1600 Average training loss: 0.3047
Epoch 2/4 Iteration: 1700 Average training loss: 0.3278
Epoch 2/4 Iteration: 1800 Average training loss: 0.3250
E

### get user embeddings

In [107]:
action_data = process_nevo_data()

In [ ]:
action_data

## BLP

# 以下测试

In [105]:
product_data = pd.read_csv(r"E:\学习\UBC\ECON622\final project\nevodata\product_data.csv")
agent_data = pd.read_csv(r"E:\学习\UBC\ECON622\final project\nevodata\agent_data.csv")


brand_dummies = pd.get_dummies(product_data['brand_ids'], prefix='brand')
x1 = pd.concat([product_data['prices'],brand_dummies],axis=1)
x2 = product_data[['prices', 'sugar', 'mushy']].copy()
x2.insert(2, 'constant', np.ones(x2.shape[0]))
s_jt = product_data['shares']

nodes = agent_data[['nodes0','nodes1','nodes2','nodes3']]
consumers = agent_data[['income','income_squared','age','child']]
new_shape = (94, 80)
nu = pd.DataFrame(np.array(nodes.values.reshape(new_shape[0], -1))[:,:new_shape[1]])
demogr = pd.DataFrame(np.array(consumers.values.reshape(new_shape[0], -1))[:,:new_shape[1]])

In [ ]:
class nevo_BLP():
    def __init__(self):
        self.n_user = 20     # 20 consumers per market
        self.n_mkt = 94      # 47 cities * 2 quarters
        self.n_product = 24  # 24 products per market
        
        self.x1 = x1
        self.x2 = x2
        self.s_jt = s_jt
        self.nu = nu
        self.demogr = 

In [70]:
from scipy.io import loadmat

In [78]:
ps2 = loadmat(r"E:\学习\UBC\ECON622\final project\nevodata\ps2.mat")

array([[ 1.        ,  0.07208794,  2.        ,  1.        ],
       [ 1.        ,  0.11417849, 18.        ,  1.        ],
       [ 1.        ,  0.13239066,  4.        ,  1.        ],
       ...,
       [ 1.        ,  0.13701741,  6.        ,  0.        ],
       [ 1.        ,  0.10017433, 12.        ,  0.        ],
       [ 1.        ,  0.12755747,  0.        ,  0.        ]])

In [103]:
ps2['demogr'].shape

(94, 80)

## step 1: prepare the data

In [1]:
import pyblp
import pandas as pd
product_data = pd.read_csv(pyblp.data.NEVO_PRODUCTS_LOCATION)
agent_data = pd.read_csv(pyblp.data.NEVO_AGENTS_LOCATION)

In [3]:
agent_data.head()

,market_ids,city_ids,quarter,weights,nodes0,nodes1,nodes2,nodes3,income,income_squared,age,child
0,C01Q1,1,1,0.05,0.434101,-1.500838,-1.151079,0.161017,0.495123,8.331304,-0.230109,-0.230851
1,C01Q1,1,1,0.05,-0.726649,0.133182,-0.500750,0.129732,0.378762,6.121865,-2.532694,0.769149
2,C01Q1,1,1,0.05,-0.623061,-0.138241,0.797441,-0.795549,0.105015,1.030803,-0.006965,-0.230851
3,C01Q1,1,1,0.05,-0.041317,1.257136,-0.683054,0.259044,-1.485481,-25.583605,-0.827946,0.769149
4,C01Q1,1,1,0.05,-0.466691,0.226968,1.044424,0.092019,-0.316597,-6.517009,-0.230109,-0.230851


In [2]:
brand_dummies = pd.get_dummies(product_data['brand_ids'], prefix='brand')
product_data = pd.concat([product_data, brand_dummies], axis=1)
product_data.to_csv(r"C:\Users\song_\Desktop\product_data_dummy.csv",index=False)

In [ ]:
class BLP():
    def __init__(self):
        product_data = pd.read_csv(pyblp.data.NEVO_PRODUCTS_LOCATION)
        agent_data = pd.read_csv(pyblp.data.NEVO_AGENTS_LOCATION)

        self.ncons = 20      # 20 simulated consumers per market
        self.nmkt = 94       # 94 markets
        self.nbrand = 24     # 24 brands per market

        self.x1 = product_data[]